# Python for Financial Analysis and Algorithmic Trading

# WARNING: 

# QUANTOPIAN is no longer active # 


__Quantopian__ (quantopian.com) was a Boston-based company that aimed to create a crowd-sourced hedge fund by letting freelance quantitative analysts develop, test, and use trading algorithms to buy and sell securities.. Its primary competitors were other open source trading platforms, mainly Numerai, QuantConnect, and WorldQuant. In November 2020, Quantopian announced it will shut down and will join Robinhood, a U.S.-based financial services firm and broker-dealer popular with millennials.

### Chapters 10 and 11 are not usable as there is no more a platform called Quantopian

# 11 - Advanced Quantopian and Trading Algorithms

We'll cover the following topics:

* Pipeline Example
* Leverage
* Hedging
* PyFolio for Portfolio Analysis
* Stock Sentiment Trading Strategy
* Futures

## Pipeline Example

We have learned how Pipeline works in the last section, but haven't explored any deeper examples.  
Our main approach will be to select a **universe** of stocks to trade from, then either go long or short, depending on some factor or filter.  

We'll start in the Research Notebook environment and then switch over to the IDE for implementation.

### Pipeline Example in RESEARCH First 

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

In [ ]:
#Q[number]US : number of US tradable stocks in included based on volume, liquidity
from quantopian.pipeline.filters import Q1500US 


Filter 1 : tradable stocks

In [ ]:
universe = Q1500US()

Filter2 : energy sector

In [ ]:
from quantopian.pipeline.data import morningstar

In [ ]:
sector = morningstar.asset_classification.morningstar_sector_code.latest

sector codes https://www.quantopian.com/docs/data-reference/morningstar_fundamentals#super-sector-sector-group-and-industry-mappings-november-2019-onward

In [ ]:
energy_sector = sector.eq(309)

masking filters help efficient coding import some factors and apply the masking

In [ ]:
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

In [ ]:
dollar_volume = AverageDollarVolume(window_length=30)

In [ ]:
# Construct a Filter matching the top N asset values of self each day.
# dollar_volume.top()

# top 10 percent based on volume
high_dollar_volume = dollar_volume.percentile_between(90, 100)

In [ ]:
# apply another filter over the masking above
top_open_prices = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)
high_close_price = USEquityPricing.close.latest.percentile_between(90,100, mask=top_open_prices)

up until now, 
we select `Q1500US` as our universe,
then apply an `sector` filter    
and then we create a mask to see high top 10 % volume stocks based on last 30 day volume    
and then we filter top 50 of the high volume stocks  
and among those we filter the top 10 % as per their close prices.

In [ ]:
def make_pipeline():
    
    # Universe Q1500US
    base_universe = Q1500US()
    
    # Energy Sector
    energy_sector = sector.eq(309)
    
    # Combine above and Make mask of 1500US and Energy
    base_energy = base_universe & energy_sector
    
    # Dollar Volume grab the info
    dollar_volume = AverageDollarVolume(window_length=30)
    
    # Grab the top 5% in avg dollar volume
    high_dollar_volume = dollar_volume.percentile_between(95, 100)
        
    # Combine the filters again
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10, mask=top_five_base_energy)
    
    # 30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30, mask=top_five_base_energy)
    
    # Percent Difference
    percent_difference = (mean_10 - mean_30) / mean_30
    
    # List of Shorts
    shorts = percent_difference < 0
    
    # List of longs 
    longs = percent_difference > 0
    
    # final Mask/filter for anything in shorts or Longs
    securities_to_trade = (shorts | longs)
    
    # Return Pipeline
    return Pipeline(columns = {
                    'longs' : longs,
                    'shorts': shorts,
                    'perc_diff': percent_difference},
                    screen= securities_to_trade
                    )
    

In [ ]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05' )

# Pipeline Execution Time: xx.00 Seconds # depending on your run

In [ ]:
result

In [ ]:
# output 
# longs	perc_diff	shorts
# 2015-05-05 00:00:00+00:00	
# Equity(216 [HES])	True	0.036528	False
# Equity(448 [APA])	True	0.035926	False
# Equity(455 [APC])	True	0.049153	False
# Equity(858 [BHI])	True	0.033807	False
# Equity(1746 [COG])	True	0.058078	False
# Equity(2368 [DVN])	True	0.046264	False
# Equity(2564 [EOG])	True	0.032102	False
# Equity(2621 [VAL])	True	0.060197	False
# Equity(3443 [HAL])	True	0.049257	False
# Equity(3647 [HP])	True	0.040991	False
# Equity(5035 [MRO])	True	0.061598	False
# Equity(5213 [NBL])	True	0.010443	False
# Equity(5214 [NBR])	True	0.064133	False
# Equity(5249 [NE])	True	0.037559	False
# Equity(5729 [OXY])	True	0.029776	False
# Equity(6928 [SLB])	True	0.046555	False
# Equity(7244 [SWN])	True	0.070788	False
# Equity(7612 [ANDV])	True	0.005997	False
# Equity(7990 [VLO])	False	-0.017145	True
# Equity(8214 [WMB])	True	0.018876	False
# Equity(8347 [XOM])	True	0.017343	False
# Equity(8461 [CHK])	True	0.014265	False
# Equity(9038 [RIG])	True	0.048180	False
# Equity(13176 [CAM])	True	0.082110	False
# Equity(17436 [PXD])	True	0.010248	False
# Equity(19249 [RRC])	True	0.087062	False
# Equity(19336 [WFT])	True	0.049141	False
# Equity(22784 [FTI])	True	0.054529	False
# Equity(23112 [CVX])	True	0.018972	False
# Equity(23998 [COP])	True	0.023902	False
# Equity(24809 [NOV])	True	0.024940	False
# Equity(25707 [WLL])	True	0.048205	False
# Equity(33856 [CLR])	True	0.064304	False
# Equity(34440 [CXO])	True	0.042184	False
# Equity(39797 [OAS])	True	0.042388	False
# Equity(40852 [KMI])	True	0.023016	False
# Equity(41636 [MPC])	True	0.011952	False
# Equity(42788 [PSX])	True	0.020911	False


### Pipeline in IDE

In [ ]:
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume, SimpleMovingAverage
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data import morningstar



def initialize(context):
    schedule_function(my_rebalance, date_rules.week_start(), time_rules.market_open(hours=1))    
    
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

    
def my_rebalance(context, data):
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            order_target_percent(security, 0)
            
    for security in context.longs:
        if data.can_trade(security):
            order_target_percent(security, context.long_weight)

    for security in context.shorts:
        if data.can_trade(security):
            order_target_percent(security, context.short_weight)

            
            
def my_compute_weights(context):
    
    if len(context.longs) == 0:
        long_weight = 0 
    else:
        long_weight = 0.5 / len(context.longs)

        
    if len(context.shorts) == 0:
        short_weight = 0 
    else:
        short_weight = 0.5 / len(context.longs)
    
    return (long_weight, short_weight)



def before_trading_start(context):
    context.output = pipeline_output('my_pipeline')
    
    #LONG
    context.longs = context.output[context.output['longs']].index.tolist()
    
    #SHORT
    context.shorts = context.output[context.output['shorts']].index.tolist()
    
    context.long_weight, context.short_weight = my_compute_weights(context)
    
    
def make_pipeline():
    
    # Universe Q1500US
    base_universe = Q1500US()
    
    # Energy Sector
    sector = morningstar.asset_classification.morningstar_sector_code.latest
    energy_sector = sector.eq(309)
    
    # Combine above and Make mask of 1500US and Energy
    base_energy = base_universe & energy_sector
    
    # Dollar Volume grab the info
    dollar_volume = AverageDollarVolume(window_length=30)
    
    # Grab the top 5% in avg dollar volume
    high_dollar_volume = dollar_volume.percentile_between(95, 100)
        
    # Combine the filters again
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10, mask=top_five_base_energy)
    
    # 30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30, mask=top_five_base_energy)
    
    # Percent Difference
    percent_difference = (mean_10 - mean_30) / mean_30
    
    # List of Shorts
    shorts = percent_difference < 0
    
    # List of longs 
    longs = percent_difference > 0
    
    # final Mask/filter for anything in shorts or Longs
    securities_to_trade = (shorts | longs)
    
    # Return Pipeline
    return Pipeline(columns = {
                    'longs' : longs,
                    'shorts': shorts,
                    'perc_diff': percent_difference},
                    screen= securities_to_trade
                    )
    

## Leverage

Leverage is the ability to borrow money for use in investing.
A simple example of leverage would be taking on a mortgage when purchasing a real estate.  

For algorithmic trading, leverage is reinvesting debt to gain a greater return on our investments.  
You would only do this if your were very confident in your strategy and believed it to be low-risk.  

In the real world, leverage would be provided by your broker, and would be highly dependent on your strategy, your account size, previous payments, fees, etc...  

Typically we can measure leverage in terms of a ratio : 

* Leverage Ratio = (Debt + Base) / Base  

The sum of your debt and capital base divided by your capital base.  


Let's explore leverage in Quantopian.  
* We'll work with both the Research Notebook and the IDE, so open up two tabs in your browser!  

* Make sure to reference the notebook for the code used here  

In [ ]:
# Leverage Algorithm in the IDE ..

def initialize(context):
    context.amzn = sid(16841)
    context.ibm = sid(3766)
    
    
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open())
    schedule_function(record_vars, date_rules.every_day(), time_rules.market_close())

def rebalance(context, data):
    order_target_percent(context.amzn, 0.5)
    order_target_percent(context.ibm, -0.5)
    
    
def record_vars(context, data):
    record(amzn_close = data.current(context.amzn, 'close'))
    record(ibm_close  = data.current(context.ibm , 'close'))
    record(Leverage = context.account.leverage)
    record(Exposure = context.account.net_leverage)

Now after running the above code in the IDE, and running a full backtest on the system we will proceed to RESEACH environment. 

This url portion was generated on the Build Algorithm phase in the IDE: notice the hash at the end.
https://www.quantopian.com/algorithms/5e80ce830ad954775d9c3f47

From the IDE, we received the below url from the backtest page. 
https://www.quantopian.com/algorithms/5e80ce830ad954775d9c3f47/5e80d033d607c645a710fbae

Notice the second url is also adding a another hash code at the end referring to the backtest. 
Let's request this again on the IDE, to deep dive from the reseach environment. 

In [ ]:
bt = get_backtest('5e80d033d607c645a710fbae')
# 100% Time:  0:00:01|##########################################################|

In [ ]:
bt
# <qexec.research.backtest.BacktestResult at 0x7f53a6223da0>

In [ ]:
bt.benchmark_security 
# 8554

In [ ]:
bt.algo_id
# '5e80d033d607c645a710fbae'

In [ ]:
bt.recorded_vars.head()

In [ ]:
bt.recorded_vars['Leverage'].plot()

## HEDGING

In this lecture we will explore hedging a bit further. 
A lot of what we discuss here is related to CAPM, so make sure to review those lectures!  


* Recall that Beta in the CAPM, represents an asset's exposure to the market(typically represented as the S&P500).  
* Trading strategies with a low Beta are attactive, as they should operate independently from the market.   

In this notebook, we will calculate an asset's Beta and Alpha and then show a simple method of hedging against the Beta, in an attempt to cancel out any risk exposure to the market.   

* We will choose a stock such as AAPL, then get its Alpha and Beta values.   
* Then we can calculate a short position on the SPY to eliminate its exposures to the market, and trade only on the asset's alpha.  

Let's open a new Resarch notebook in Quantopian.

In [ ]:
import numpy as np
from statsmodels import regression
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
start = '2016-01-01'
end   = '2017-01-01'

In [ ]:
asset = get_pricing('AAPL', fields='price', start_date=start, end_date=end)

In [ ]:
bench = get_pricing('SPY', fields='price', start_date=start, end_date=end)

In [ ]:
asset_ret = asset.pct_change(1)[1:]
bench_ret = bench.pct_change(1)[1:]

Plot

In [ ]:
asset_ret.plot()
bench_ret.plot()
plt.legend()

In [ ]:
plt.scatter(bench_ret, asset_ret, alpha=0.6, s=50)
plt.xlabel('SPY Ret')
plt.xlabel('AAPL Ret')

#### get alpha and beta through regression

In [ ]:
AAPL = asset_ret.values
spy = bench_ret.values

In [ ]:
spy_constant = sm.add_constant(spy)

In [ ]:
spy_constant;

In [ ]:
model = regression.linear_model.OLS(AAPL, spy_constant).fit()

In [ ]:
model.params

In [ ]:
alpha, beta = model.params

In [ ]:
print('alpha  :' + str(alpha))
print('beta  :' + str(alpha))

In [ ]:
min_spy = bench_ret.values.min()
max_spy = bench_ret.values.max()

spy_line = np.linspace(min_spy, max_spy, 100)
y = spy_line*beta + alpha

plt.plot(spy_line,  y, 'r')
plt.scatter(bench_ret, asset_ret, alpha=0.6, s=50)
plt.xlabel('SPY Ret')
plt.ylabel('AAPL Ret')

In [ ]:
# now lets implement a simple hedge. 
# short the market by beta

hedged  = (-1) * (beta * bench_ret) + asset_ret

In [ ]:
hedged.plot(label='AAPL with Hedge')
asset_ret.plot(alpha=0.5)
bench_ret.plot(alpha=0.5)
plt.xlim(['2016-06-01', '2016-08-01'])
plt.legend()

In [ ]:
def alpha_beta(benchmark_ret, stock):
    
    benchmark = sm.add_constant(benchmark_ret)
    
    model =regression.linear_model.OLS(stock, benchmark).fit()
    
    return model.params[0], model.params[1]

In [ ]:
## 2016 VALUES
# alpha 1.679902481e-05
# beta 1.02981369795

In [ ]:
portfolio = -1 * beta2016 * benchmark_ret2016 + asset_ret2016

In [ ]:
alpha, beta = alpha_beta(benchmark_ret2016, portfolio)

In [ ]:
print('PORTFOLIO ALPHA AND BETA')

print('alpha  ' + str(alpha))
print('beta   ' + str(beta))

In [ ]:
# PORTFOLIO ALPHA AND BETA
# alpha  1.679902481e-05
# beta   1.36609473733e-16

# Notice above beta is close to zero, meaning it is nearly eliminated.

In [ ]:
portfolio.plot(alpha=0.9, label='AAPL with HEDGE')
asset_ret2016.plot(alpha=0.5)
benchmark_ret2016.plot(alpha=0.5)
plt.ylabel('DAILY RETURN')
plt.legend()

In [ ]:
print('portfolio mean    :' + str(portfolio.mean()))
print('asset_ret2016 mean:' + str(asset_ret2016.mean()))

print('-------------------------------------------')

print('portfolio std     :' + str(portfolio.std()))
print('asset_ret2016 std :' + str(asset_ret2016.std()))

In [ ]:
# Comparing mean returns and std's 

# portfolio mean    :0.0011399935632582228
# asset_ret2016 mean:0.000574549568770769
# -------------------------------------------
# portfolio std     :0.009066375411652783
# asset_ret2016 std :0.014702747344261722

# both mean returns and std's are now less than the original values

#### Let's do it in the 2017 first 8 months this time. 

In [ ]:
# 2017 CELL

start = '2017-01-01'
end   = '2017-08-01'
asset2017 = get_pricing('AAPL', fields='price', start_date=start, end_date=end)
benchmark2017 = get_pricing('SPY', fields='price', start_date=start, end_date=end)

asset_ret2017 = asset2017.pct_change(1)[1:]
benchmark_ret2017 = benchmark2017.pct_change(1)[1:]

aret_values = asset_ret2017.values
bret_values = benchmark_ret2017.values

alpha2017, beta2017 = alpha_beta(bret_values, aret_values)

print('2017 Based Figures')
print('alpha ' + str(alpha2017))
print('beta ' + str(beta2017))

In [ ]:
# 2017 Based Figures
# alpha 0.000968854542346
# beta 1.26769880296

In [ ]:
print(alpha2016)
print(beta2016)

In [ ]:
# 1.679902481e-05
# 1.02981369795

In [ ]:
# we are using previous years beta in today's information
portfolio  =  -1 * beta2016 * benchmark_ret2017 + asset_ret2017

In [ ]:
alpha, beta = alpha_beta(benchmark_ret2017, portfolio)

In [ ]:
print('portfolio mean    :' + str(portfolio.mean()))
print('asset_ret2017 mean:' + str(asset_ret2017.mean()))

print('-------------------------------------------')

print('portfolio std     :' + str(portfolio.std()))
print('asset_ret2017 std :' + str(asset_ret2017.std()))

In [ ]:
# portfolio mean    :0.0011399935632582228
# asset_ret2017 mean:0.0018808609159293456
# -------------------------------------------
# portfolio std     :0.009066375411652783
# asset_ret2017 std :0.010484461271996029

We are sacrificing average returns for less volatility, lowering both mean returns and volatility.

## Portfolio Analysis with PYFOLIO

Quantopian has a built in library called **PyFolio** that quickly creates "tear-sheets" and useful information plots easily. Let's explore this!

https://github.com/quantopian/pyfolio

In [ ]:
# Following code section to be run on Quantopian

import pyfolio as pf
import matplotlib.pyplot as plt
import empyrical

In [ ]:
# algo returns
# use leverage exercise backtest.
bt = get_backtest('5e80d033d607c645a710fbae')

In [ ]:
bt_returns = bt.daily_performance['returns']

In [ ]:
bt_positions = bt.pyfolio_positions

In [ ]:
bt_transactions = bt.pyfolio_transactions

In [ ]:
empyrical.sharpe_ratio(bt_returns)

In [ ]:
# bt.create_full_tear_sheet();

___

In [4]:
# Benchmark returns from the SPY leverage  hash :  5e81269e32fc8d45ae5468e0

benchmark_rets = get_backtest('5e81269e32fc8d45ae5468e0')

In [ ]:
bm_returns =  benchmark_rets.daily_performance['returns']

In [ ]:
bm_positions = benchmark_rets.pyfolio_positions
bm_transactions = benchmark_rets.pyfolio_transactions

In [ ]:
print('Sharp Ratio for portfolio :', str(empyrical.sharpe_ratio(bt_returns)))
print('Sharp Ratio for benchmark :', str(empyrical.sharpe_ratio(bm_returns)))

# Sharp Ratio for portfolio : 0.012790682682849846
# Sharp Ratio for benchmark : 0.4482604408899276

In [ ]:
# pf.plotting

In [ ]:
#  Cumulative Returns
plt.subplot(2,1,1)
pf.plotting.plot_rolling_returns(bt_returns, bm_returns)

# Daily non-cumulative returns
plt.subplot(2,1,2)
pf.plotting.plot_returns(bt_returns)
plt.tight_layout()

In [ ]:
fig = plt.figure(1)
plt.subplot(1,3,1)
pf.plot_annual_returns(bt_returns)

plt.subplot(1,3,2)
pf.plot_monthly_returns_dist(bt_returns)

plt.subplot(1,3,3)
pf.plot_monthly_returns_heatmap(bt_returns)

plt.tight_layout()
fig.set_size_inches(15,5)

In [ ]:
pf.plot_return_quantiles(bt_returns)

In [ ]:
pf.plot_rolling_beta(bt_returns, bm_returns);

In [ ]:
pf.plot_rolling_sharpe(bt_returns);

In [ ]:
pf.plot_drawdown_periods(bt_returns);

In [ ]:
pf.plot_drawdown_underwater(bt_returns);

In [ ]:
pf.plot_gross_leverage(bt_returns, bt_positions)

In [ ]:
pf.create_round_trip_tear_sheet(bt_returns, bt_positions, bt_transactions)

## Stock Sentiment Analysis

* We have currently looked at numerical data as a trading signal, but there is a lot more information out in the world!
* A common source for signal is to use sentiment analysis

Sentiment Analysis uses natural language processing to attempt to detect sentiment in some text.   
ie. Positive vs Negative Sentiments.  

This sort of data is almost never free.  Quantopian does have a free version of sentiment analysis (available only for years 2013-2015).  Accern Alphaone provides a sentiment from -1 to 1 and an impact score of 0 to 100.

**Impact** is a measure of how likely a stock price will change as a result of the sentiment. 
In the following lectures, we will show you how to access this free data and setup a trading strategy with it!  

In [6]:
# No Code for Alphaone 
# use sentdex instead.
# https://www.quantopian.com/docs/data-reference/sentdex#sentdex-data-reference

In [ ]:
from quantopian.pipeline.data.sentdex import sentiment
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.research import run_pipeline

# Define a 5-day simple moving average sentiment factor.
mean_sentiment_5day = SimpleMovingAverage(inputs=[sentiment.sentiment_signal], window_length=5)

# Add the sentiment factor to a pipeline.
pipe = Pipeline(
    columns={
        'mean_sentiment_5day': mean_sentiment_5day,
    },
    domain=US_EQUITIES,
)

# Run the pipeline for a year and print the first few rows of the result.
df = run_pipeline(pipe, '2017-05-05', '2018-05-05')
print(df.head())

## Futures

Extra Resources:

https://en.wikipedia.org/wiki/Futures_contract
http://www.investopedia.com/terms/f/futurescontract.asp

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Getting Futures
A reference to a futures contract is obtained via the symbols function. Futures contracts are denoted by a base symbol + a code for month/year of delivery. CLF16 is a contract for crude oil (CL) with delivery in January (F) 2016 (16).

List of all Available Futures (on Quantopian)


https://www.quantopian.com/docs/api-reference/algorithm-api-reference#futures  



Let's grab the future contract data for Natural Gas for a maturity date of January 2018. (If you are viewing this lecture some time in the future, choose a further out maturity date)

In [ ]:
future_contract = symbols('NGF18')

In [ ]:
future_contract.asset_name

# u'Natural Gas'

In [ ]:
for key in future_contract.to_dict():
    print(key,'\t', future_contract.to_dict()[key] )

In [ ]:
futures_position_value = get_pricing(future_contract, start_date = '2017-01-01', end_date = '2018-01-01', fields = 'price')
futures_position_value.name = futures_position_value.name.symbol
futures_position_value.plot()
plt.title('NG Futures Price')
plt.xlabel('Date')
plt.ylabel('Price');

In [ ]:
from quantopian.research.experimental import history
print history.__doc__

In [ ]:
Load a table of historical trade data.

    Parameters
    ----------
    symbols : Asset-convertible object, ContinuousFuture, or iterable of same.
        Valid input types are Asset, Integral, basestring, or ContinuousFuture.
        In the case that the passed objects are strings, they are interpreted
        as ticker symbols and resolved relative to the date specified by
        symbol_reference_date.

    fields : str or list
        String or list drawn from {'price', 'open_price', 'high', 'low',
        'close_price', 'volume', 'contract'}.

    start : str or pd.Timestamp
        String or Timestamp representing a start date or start intraday minute
        for the returned data.

    end : str or pd.Timestamp
        String or Timestamp representing an end date or end intraday minute for
        the returned data.

    frequency : {'daily', 'minute'}
        Resolution of the data to be returned.

    symbol_reference_date : str or pd.Timestamp, optional
        String or Timestamp representing a date used to resolve symbols that
        have been held by multiple companies. Defaults to the current time.

    handle_missing : {'raise', 'log', 'ignore'}, optional
        String specifying how to handle unmatched securities. Defaults to
        'raise'.

    start_offset : int, optional
        Number of periods before ``start`` to fetch.
        Default is 0. This is most often useful when computing returns.

    Returns
    -------
    pandas Panel/DataFrame/Series
        The pricing data that was requested. See note below.

    Notes
    -----
    If a list of symbols is provided, data is returned in the form of a pandas
    Panel object with the following indices::

        items = fields
        major_axis = TimeSeries (start_date -> end_date)
        minor_axis = symbols

    If a string is passed for the value of `symbols` and `fields` is None or a
    list of strings, data is returned as a DataFrame with a DatetimeIndex and
    columns given by the passed fields.

    If a list of symbols is provided, and `fields` is a string, data is
    returned as a DataFrame with a DatetimeIndex and a columns given by the
    passed `symbols`.

    If both parameters are passed as strings, data is returned as a Series.
    

In [ ]:
ngf18 = future_contract

In [ ]:
ngf18_data = history(ngf18, 
                     fields=['price', 'open_price', 'high', 'low', 'close_price', 'volume', 'contract'], 
                     frequency='daily', 
                     start='2017-06-01', 
                     end='2017-08-01')

In [ ]:
# Notice the 4th of July!
ngf18_data['volume'].plot(kind='bar')

### Comparison of Different Maturity Dates

In [ ]:
ng_contracts = symbols(['NGF17', 'NGG17', 'NGH17', 'NGJ17', 'NGK17', 'NGM17'])

In [ ]:
ng_consecutive_contract_volume.plot()

In [ ]:
ng_consecutive_contract_volume.plot(xlim=['2016-10-01','2017-08-01'])

Trading activity jumps from one contract to the next. Transitions happen just prior to the delivery date of each contract.

This phenomenon can make it difficult to work with futures. Having to explicitly reference a series of transient contracts when trading or simulating futures can be a hassle.

In order to trade consecutive contracts for the same underlying future, we can use what's called a "Continuous Future".


###  Continuous Futures


Continuous futures are abstractions over the 'underlying' commodities/assets/indexes of futures. For example, if we wanted to trade crude oil, we could create a reference to CL, instead of a series of CL contracts. Continuous futures essentially maintain a reference to a 'current' contract deemed to be the active contract for the particular underlying.

We use the continuous futures objects as part of the platform to get a continuous chain of historical data for futures contracts, taking these concerns into account. There are several ways to adjust for the cost of carry when looking at historical data, though people differ on what they prefer. The general consensus is that an adjustment should be done.

Continuous futures are not tradable assets. They maintain a reference to the current active contract related to a given underlying.

In [ ]:
from quantopian.research.experimental import continuous_future

In [ ]:
print(continuous_future.__doc__)

There are 4 arguments that we need to consider.

**root_symbol**: The root symbol of the underlying. For example, 'CL' for crude oil.  
**offset**: The distance from the primary contract. 0 = primary, 1 = secondary, etc. We'll get into this more later.  
**roll**: How to determine the 'current' contract of the continuous future. Current options are 'volume' and 'calendar'. The **volume**' approach chooses the current active contract based on trading volume.   
The '**calendar'** approach chooses the current active contract based simply on the auto_close_dates of each contract.**  
adjustment: How to adjust historical prices from earlier contracts.   


We'll get into this more later. Options are 'mul', 'add', or 'None'.


In [ ]:
continuous_ng = continuous_future('NG', offset=0, roll='volume', adjustment='mul')
continuous_ng
# ContinuousFuture(94090707563773952, root_symbol='NG', offset=0, roll_style='volume', adjustment='mul')

In [ ]:
ng_cont_active = history(continuous_ng, 
                    fields=['contract','price','volume'] ,
                    frequency='daily', 
                    start_date='2016-10-01', 
                    end_date='2017-08-01')

In [ ]:

ng_cont_active.head()

In [ ]:
ng_cont_active['price'].plot()

In [ ]:

ng_cont_active['volume'].plot()

In [ ]:
ng_consecutive_contract_volume = history(ng_contracts, 
                                         fields='volume', 
                                         frequency='daily', 
                                         start_date='2016-10-01', 
                                         end_date='2017-08-01')

In [ ]:
ax = ng_cont_active['volume'].plot(ls='--',c='black',lw=3)
ng_consecutive_contract_volume.plot(ax=ax)

In [ ]:
ng_consecutive_contract_price = history(ng_contracts, 
                                         fields='price', 
                                         frequency='daily', 
                                         start_date='2016-10-01', 
                                         end_date='2017-08-01')

In [ ]:
ng_continuous_active = history(continuous_ng, 
                    fields=['contract','price','volume'] ,
                    frequency='daily', 
                    start_date='2016-10-01', 
                    end_date='2017-08-01')

In [ ]:
ng_consecutive_contract_price.plot()

In [ ]:
ng_cont_active['price'].plot(c='black',lw=3)

This represents the price of the underlying commodity, NG, on the most actively traded contract. Much easier to look at.

You might notice that the price at the start of this plot exceeds 4.0, but when we plotted the individual contracts, it barely made it above 3.6. This is because the historical price is getting adjusted for jumps between contracts.

The best way to explain this is to plot the prices history of the unadjusted continuous future.


In [ ]:
continuous_ng_unadjusted = continuous_future('NG', offset=0, roll='volume', adjustment=None)

In [ ]:
ng_unadjusted_history = history(continuous_ng_unadjusted, 
                                fields=['contract', 'price'], 
                                frequency='daily', 
                                start_date='2016-10-01', 
                                end_date='2017-08-01')

In [ ]:
ng_unadjusted_history.head()

In [ ]:
ng_unadjusted_history.plot()

In [ ]:

ng_consecutive_contract_price.plot()

In [ ]:
pivot_unadj = ng_unadjusted_history.pivot(index=ng_unadjusted_history.index,columns='contract')

pivot_unadj.head()

In [ ]:
pivot_unadj.plot()

In [ ]:
ax = pivot_unadj.plot()
ng_unadjusted_history.plot(ax=ax,ls='--',c='black')

### Adjustment Types
There are two main adjustment types, additive or multiplicative.

* Multiplicative adjustment, mul  
This essentially computes the adjustment as the ratio of new contract price / old contract price whenever the active contract rolls to a new contract.

* Arithmetic adjustment, 'add'   
The 'add' technique computes the adjustment as the difference new contract price - old contract price.

# *** THE END ***